In [1]:
import os
import json
import h5py
import numpy as np

import robomimic
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.obs_utils as ObsUtils
import imageio
import matplotlib.pyplot as plt
import cv2 

### write good,bad segment on the video frames

In [2]:
dataset_path = "/home/ns1254/robomimic/datasets/can/mh/image_v141+groups.hdf5"
# dataset_path = "/home/ns1254/data_robomimic/carl_dataset/mixed_human_original/can_carl_mix_v3_510.hdf5"

In [3]:
f = h5py.File(dataset_path, "r")
demos = list(f["data"].keys())
num_demos = len(demos)
print("hdf5 file {} has {} demonstrations".format(dataset_path, num_demos))

hdf5 file /home/ns1254/robomimic/datasets/can/mh/image_v141+groups.hdf5 has 300 demonstrations


In [4]:
f['mask'].keys()

<KeysViewHDF5 ['20_percent', '20_percent_train', '20_percent_valid', '50_percent', '50_percent_train', '50_percent_valid', 'better', 'better+w10', 'better_operator_1', 'better_operator_1_train', 'better_operator_1_valid', 'better_operator_2', 'better_operator_2_train', 'better_operator_2_valid', 'better_train', 'better_valid', 'okay', 'okay_better', 'okay_better_train', 'okay_better_valid', 'okay_operator_1', 'okay_operator_1_train', 'okay_operator_1_valid', 'okay_operator_2', 'okay_operator_2_train', 'okay_operator_2_valid', 'okay_train', 'okay_valid', 'train', 'valid', 'w1b1', 'worse', 'worse_better', 'worse_better_train', 'worse_better_valid', 'worse_okay', 'worse_okay_train', 'worse_okay_valid', 'worse_operator_1', 'worse_operator_1_train', 'worse_operator_1_valid', 'worse_operator_2', 'worse_operator_2_train', 'worse_operator_2_valid', 'worse_train', 'worse_valid']>

In [5]:
f.keys()
is_normalized = 'mins' in f.keys()
action_mins = None
action_maxs = None
if is_normalized:
    action_mins=np.array( f['mins'] )
    action_maxs=np.array(f['maxs'])

is_normalized

False

In [6]:
env_meta = json.loads(f["data"].attrs["env_args"])
# env_meta = FileUtils.get_env_metadata_from_dataset(dataset_path=dataset_path)

env = EnvUtils.create_env_from_metadata(
    env_meta=env_meta, 
    render=False,            # no on-screen rendering
    render_offscreen=True,   # off-screen rendering to support rendering video frames
)

[robosuite WARNING] No private macro file found! (macros.py:53)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)
[robosuite WARNING] To setup, run: python /home/ns1254/robosuite/robosuite/scripts/setup_macros.py (macros.py:55)


Created environment with name PickPlaceCan
Action size is 7


In [7]:
# dummy spec necessary for playing back
dummy_spec = dict(
    obs=dict(
            low_dim=["robot0_eef_pos"],
            rgb=["agentview_image", "robot0_eye_in_hand"],
        ),
)
ObsUtils.initialize_obs_utils_with_obs_specs(obs_modality_specs=dummy_spec)


============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['robot0_eef_pos']
using obs modality: rgb with keys: ['robot0_eye_in_hand', 'agentview_image']


In [9]:
# task="can" 
# dataset_type="510" 
# group="p20b"
# more="gb_marked3"
# tosave='video/'+task+'/'+dataset_type+'/'+group+'_'+more
# print('tosave:',tosave)
# if not os.path.exists(tosave):
#     os.makedirs(tosave)

tosave: video/can/510/p20b_gb_marked3


In [8]:
task="can" 
dataset_type="mh" 
group="w1b1"
more="gb_marked_m7"
tosave='video/'+task+'/'+dataset_type+'/'+group+'_'+more
print('tosave:',tosave)
if not os.path.exists(tosave):
    os.makedirs(tosave)

tosave: video/can/mh/w1b1_gb_marked_m7


In [9]:
# demos=[b  for b in f['data'].keys()]  
demos=[b.decode('utf-8')  for b in f['mask'][group]]  
len(demos), demos[:3]

(100, ['demo_0', 'demo_1', 'demo_10'])

In [10]:
# fn="/home/ns1254/icra2025/segments_data_s2.json"
# fn="/home/ns1254/icra2025/segments_data_s5_can510_p20b.json"
fn="/home/ns1254/icra2025/segments_data_mh7.json"
segments_bd=json.load(open(fn))

In [11]:
segments_bd.keys()

dict_keys(['demo_200', 'demo_201', 'demo_202', 'demo_203', 'demo_204', 'demo_205', 'demo_206', 'demo_208', 'demo_209', 'demo_210', 'demo_211', 'demo_214', 'demo_216', 'demo_217', 'demo_219', 'demo_220', 'demo_221', 'demo_226', 'demo_228', 'demo_229', 'demo_231', 'demo_233', 'demo_234', 'demo_236', 'demo_243', 'demo_244', 'demo_245', 'demo_247', 'demo_248'])

In [12]:
def which_gb(demo_name, i):
    segs_g=segments_bd[demo_name]['good']
    segs_b=segments_bd[demo_name]['bad']

    gb=-1 # 0 good, 1 bad
    for seg in segs_g:
        if i>=seg[0] and i<=seg[1]:
            gb=0
            break
    for seg in segs_b:
        if i>=seg[0] and i<=seg[1]:
            gb=1
            break
    return gb

In [15]:
gb_bad={'demo_200': [(351, 364), (519, 540)],
 'demo_201': [(276, 295), (391, 416)],
 'demo_202': [(261, 302), (370, 396)],
 'demo_206': [(257, 271), (275, 293), (306, 320)],
 'demo_208': [(555, 572), (608, 674)],
 'demo_209': [(248, 270)],
 'demo_214': [(119, 138)],
 'demo_217': [(481, 499), (546, 563), (580, 613)],
 'demo_221': [(127, 145)],
 'demo_228': [(142, 159)],
 'demo_229': [(250, 273), (276, 298), (328, 371)],
 'demo_244': [(75, 107), (149, 163)],
 'demo_245': [(207, 235)],
 'demo_248': [(353, 370), (403, 452)]}

In [12]:
def in_bad_segment(demo_name, i):
    if demo_name not in gb_bad:
        return False
    
    gb=False
    segs_b=gb_bad[demo_name]  
    for seg in segs_b:
        if i>=seg[0] and i<=seg[1]:
            gb=True
            break
    return gb

In [11]:
gb_bad={
'demo_134': [(210,320)],
'demo_331': [(600,1200)],
'demo_332': [(210,500), (900,1200)],
'demo_335': [(50,400)],
'demo_336': [(100,600)],
'demo_342': [(300,1200)],
'demo_343': [(100,1100)],
'demo_364': [(100,550), (1000,1100)],
'demo_365': [(450,550)],
'demo_366': [(600,1000)],
'demo_367': [(600,1200)],
'demo_368': [(500,1200)],
'demo_373': [(500,900)],
'demo_374': [(1100,1200)]
}

In [15]:
in_bad_segment('demo_0', 100) , in_bad_segment('demo_373', 550)

(False, True)

In [38]:
gb_good={'demo_200': [],
 'demo_201': [(203, 246), (422, 488)],
 'demo_202': [(190, 209)],
 'demo_204': [],
 'demo_205': [],
 'demo_206': [],
 'demo_208': [(234, 382), (487, 501)],
 'demo_209': [(275, 307)],
 'demo_211': [(296, 355)],
 'demo_214': [],
 'demo_217': [(372, 390)],
 'demo_220': [(232, 269)],
 'demo_221': [(159, 208)],
 'demo_226': [],
 'demo_228': [(179, 193), (290, 339)],
 'demo_229': [(143, 164)],
 'demo_244': [],
 'demo_245': [],
 'demo_247': [],
 'demo_248': [(182, 237), (468, 525)]}

In [14]:
def in_good_segment(demo_name, i):
    if demo_name not in gb_good:
        return False
    
    gb=False
    segs_b=gb_good[demo_name]  
    for seg in segs_b:
        if i>=seg[0] and i<=seg[1]:
            gb=True
            break
    return gb

In [15]:
demo_name="demo_249"
demo_name="demo_204"
demo_name="demo_248"
demo_name="demo_340"
which_gb(demo_name, 470)

0

In [11]:
#p20b good segments from bad
gb_good={'demo_134': [(0, 25), (168, 187)],
 'demo_136': [(0, 23)],
 'demo_331': [(0, 49)],
 'demo_332': [(0, 28)],
 'demo_333': [(404, 433), (491, 518), (634, 684), (708, 742)],
 'demo_335': [(0, 18)],
 'demo_336': [],
 'demo_337': [(0, 19), (191, 228)],
 'demo_338': [(868, 931), (968, 989), (1013, 1081), (1157, 1199)],
 'demo_339': [(826, 892)],
 'demo_340': [(0, 24), (450, 472), (597, 626)],
 'demo_341': [],
 'demo_342': [(0, 29)],
 'demo_343': [(0, 24)],
 'demo_344': [(584, 637)],
 'demo_345': [],
 'demo_361': [(1027, 1093)],
 'demo_362': [(0, 33), (469, 532)],
 'demo_363': [(345, 361),
  (441, 484),
  (504, 550),
  (743, 770),
  (940, 976),
  (999, 1040),
  (1135, 1155)],
 'demo_364': [(0, 42)],
 'demo_365': [(469, 598)],
 'demo_366': [(0, 21)],
 'demo_367': [(0, 78)],
 'demo_368': [(0, 22), (247, 266), (420, 452)],
 'demo_369': [(679, 699), (714, 743)],
 'demo_371': [(900, 949)],
 'demo_372': [(0, 23), (951, 987)],
 'demo_373': [],
 'demo_374': [(1010, 1099)]}

In [13]:
demo_name="demo_340"
in_good_segment(demo_name, 470)

0

In [15]:
demo_name="demo_340"
in_good_segment(demo_name, 470)

True

In [16]:
in_good_segment("demo_373", 470)

False

In [18]:
in_good_segment("demo_371", 470) , in_good_segment("demo_371", 920)

(False, True)

In [13]:
def playback_and_save(demo_key):
    video_path = os.path.join(tosave, demo_key+".mp4")
    video_writer = imageio.get_writer(video_path, fps=20)
    
    init_state = f["data/{}/states".format(demo_key)][0]
    model_xml = f["data/{}".format(demo_key)].attrs["model_file"]
    initial_state_dict = dict(states=init_state, model=model_xml)

    # reset to initial state
    env.reset_to(initial_state_dict)

    # playback actions one by one, and render frames
    actions = f["data/{}/actions".format(demo_key)][:]
    
    if is_normalized:
        ac=np.array(actions)
        ac_unnorm = action_mins +( (ac+1.0)/2.0 )*(action_maxs-action_mins)
        actions = ac_unnorm
    
    for t in range(actions.shape[0]):
        env.step(actions[t])
        video_img = env.render(mode="rgb_array", height=512, width=512, camera_name="agentview")
        video_img = np.ascontiguousarray(video_img, dtype=np.uint8)
        cv2.putText(video_img, str(t), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)   #write timestep on image 
        gb=which_gb(demo_key, t)
        # gb=in_good_segment(demo_key, t)
        # gb = not in_bad_segment(demo_key, t)
        # if gb:
        #     gb=0
        # else:
        #     gb=1
        
        sgb='skip' 
        color_gb=(0, 120, 120)
        if gb==0:
            sgb='good'
            color_gb=(0, 255, 0)
        elif gb==1:
            sgb='bad'
            color_gb=(255, 0, 0)

        cv2.putText(video_img, sgb, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, color_gb, 2, cv2.LINE_AA)   #write gb info on image
        video_writer.append_data(video_img)

    video_writer.close()
    print('saved to',video_path)

In [14]:
# demo_name="demo_249"
# playback_and_save(demo_name)

In [15]:
for demo_name in segments_bd.keys():
    print(f'#demo {demo_name}')
    playback_and_save(demo_name)

#demo demo_200
saved to video/can/mh/w1b1_gb_marked_m7/demo_200.mp4
#demo demo_201
saved to video/can/mh/w1b1_gb_marked_m7/demo_201.mp4
#demo demo_202
saved to video/can/mh/w1b1_gb_marked_m7/demo_202.mp4
#demo demo_203
saved to video/can/mh/w1b1_gb_marked_m7/demo_203.mp4
#demo demo_204
saved to video/can/mh/w1b1_gb_marked_m7/demo_204.mp4
#demo demo_205
saved to video/can/mh/w1b1_gb_marked_m7/demo_205.mp4
#demo demo_206
saved to video/can/mh/w1b1_gb_marked_m7/demo_206.mp4
#demo demo_208
saved to video/can/mh/w1b1_gb_marked_m7/demo_208.mp4
#demo demo_209
saved to video/can/mh/w1b1_gb_marked_m7/demo_209.mp4
#demo demo_210
saved to video/can/mh/w1b1_gb_marked_m7/demo_210.mp4
#demo demo_211
saved to video/can/mh/w1b1_gb_marked_m7/demo_211.mp4
#demo demo_214
saved to video/can/mh/w1b1_gb_marked_m7/demo_214.mp4
#demo demo_216
saved to video/can/mh/w1b1_gb_marked_m7/demo_216.mp4
#demo demo_217
saved to video/can/mh/w1b1_gb_marked_m7/demo_217.mp4
#demo demo_219
saved to video/can/mh/w1b1_gb_mar

In [19]:
for demo_name in gb_bad.keys():
    print(f'#demo {demo_name}')
    playback_and_save(demo_name)

#demo demo_134
saved to video/can/510/p20b_gb_marked3/demo_134.mp4
#demo demo_331
saved to video/can/510/p20b_gb_marked3/demo_331.mp4
#demo demo_332
saved to video/can/510/p20b_gb_marked3/demo_332.mp4
#demo demo_335
saved to video/can/510/p20b_gb_marked3/demo_335.mp4
#demo demo_336
saved to video/can/510/p20b_gb_marked3/demo_336.mp4
#demo demo_342
saved to video/can/510/p20b_gb_marked3/demo_342.mp4
#demo demo_343
saved to video/can/510/p20b_gb_marked3/demo_343.mp4
#demo demo_364
saved to video/can/510/p20b_gb_marked3/demo_364.mp4
#demo demo_365
saved to video/can/510/p20b_gb_marked3/demo_365.mp4
#demo demo_366
saved to video/can/510/p20b_gb_marked3/demo_366.mp4
#demo demo_367
saved to video/can/510/p20b_gb_marked3/demo_367.mp4
#demo demo_368
saved to video/can/510/p20b_gb_marked3/demo_368.mp4
#demo demo_373
saved to video/can/510/p20b_gb_marked3/demo_373.mp4
#demo demo_374
saved to video/can/510/p20b_gb_marked3/demo_374.mp4


In [21]:
for demo_name in gb_good.keys():
    print(f'#demo {demo_name}')
    playback_and_save(demo_name)

#demo demo_134
saved to video/can/510/p20b_gb_marked2/demo_134.mp4
#demo demo_136
saved to video/can/510/p20b_gb_marked2/demo_136.mp4
#demo demo_331
saved to video/can/510/p20b_gb_marked2/demo_331.mp4
#demo demo_332
saved to video/can/510/p20b_gb_marked2/demo_332.mp4
#demo demo_333
saved to video/can/510/p20b_gb_marked2/demo_333.mp4
#demo demo_335
saved to video/can/510/p20b_gb_marked2/demo_335.mp4
#demo demo_336
saved to video/can/510/p20b_gb_marked2/demo_336.mp4
#demo demo_337
saved to video/can/510/p20b_gb_marked2/demo_337.mp4
#demo demo_338
saved to video/can/510/p20b_gb_marked2/demo_338.mp4
#demo demo_339
saved to video/can/510/p20b_gb_marked2/demo_339.mp4
#demo demo_340
saved to video/can/510/p20b_gb_marked2/demo_340.mp4
#demo demo_341
saved to video/can/510/p20b_gb_marked2/demo_341.mp4
#demo demo_342
saved to video/can/510/p20b_gb_marked2/demo_342.mp4
#demo demo_343
saved to video/can/510/p20b_gb_marked2/demo_343.mp4
#demo demo_344
saved to video/can/510/p20b_gb_marked2/demo_344

### save all bad segments in a single video file with slow fps

In [20]:
video_path = os.path.join(tosave, 'all_bad_segs2'+".mp4")
video_writer = imageio.get_writer(video_path, fps=5)

In [21]:
for demo_key in segments_bd.keys():
    print(f'#demo {demo_key}')
    init_state = f["data/{}/states".format(demo_key)][0]
    model_xml = f["data/{}".format(demo_key)].attrs["model_file"]
    initial_state_dict = dict(states=init_state, model=model_xml)

    # reset to initial state
    env.reset_to(initial_state_dict)

    # playback actions one by one, and render frames
    actions = f["data/{}/actions".format(demo_key)][:]
    
    if is_normalized:
        ac=np.array(actions)
        ac_unnorm = action_mins +( (ac+1.0)/2.0 )*(action_maxs-action_mins)
        actions = ac_unnorm
    
    for t in range(actions.shape[0]):
        env.step(actions[t])
        video_img = env.render(mode="rgb_array", height=512, width=512, camera_name="agentview")
        video_img = np.ascontiguousarray(video_img, dtype=np.uint8)
        cv2.putText(video_img, str(t), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)   #write timestep on image 
        gb=which_gb(demo_key, t)
        if gb==1:
            sgb='bad'
            color_gb=(255, 0, 0) 
            cv2.putText(video_img, sgb, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, color_gb, 2, cv2.LINE_AA)   #write gb info on image
            cv2.putText(video_img, demo_key, (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, color_gb, 2, cv2.LINE_AA)

            video_writer.append_data(video_img)

video_writer.close()
print('saved to',video_path)

#demo demo_200
#demo demo_201
#demo demo_202
#demo demo_204
#demo demo_205
#demo demo_206
#demo demo_208
#demo demo_209
#demo demo_211
#demo demo_214
#demo demo_217
#demo demo_220
#demo demo_221
#demo demo_226
#demo demo_228
#demo demo_229
#demo demo_244
#demo demo_245
#demo demo_247
#demo demo_248
saved to video/can/mh/w1b1_gb_marked2/all_bad_segs2.mp4


In [ ]:
# for i, demo_key in enumerate(demos):
#     print(f'#demo {i+1}/{len(demos)} {demo_key}')
#     playback_and_save(demo_key)